In [1]:
import mne
import warnings
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import FastICA
from sklearn.neural_network import MLPClassifier

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.preprocessing import ICA

from utils.Pipeline import FourierTransform, Float64CSP, EEGStandardScaler, WaveletTransform
from utils.Preprocessing import load_and_preprocess_data


In [2]:
mne.set_log_level("CRITICAL")

# Suppress warnings from LogisticRegression and CSP
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.linear_model._logistic')
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mne')

In [3]:
def model_pipeline(X_train_flattened, y_train,
                   X_test_flattened, y_test):

    pipeline = Pipeline([
        ('scaler', StandardScaler()),               
        ('pca', PCA(n_components=10)),
        ('mlp', MLPClassifier(hidden_layer_sizes=(500, 50), max_iter=100))
    ])

    pipeline.fit(X_train_flattened, y_train)

    train_accuracy = pipeline.score(X_train_flattened, y_train)
    test_accuracy = pipeline.score(X_test_flattened, y_test)
    cv_scores = cross_val_score(pipeline, X_train_flattened, y_train, cv=5)

    cv_score = np.mean(cv_scores)

    return test_accuracy, cv_score

## Run experiments

In [4]:
# Define subjects and runs for training and testing
train_subjects = [57, 22, 86]
train_tasks = {
    'Task 1:"Open and close fist"': [3, 7, 11],
    'Task 2:"Imagine opening and closing fist"': [4, 8, 12],
    'Task 3:"Open and close fists or feet"': [5, 9, 13],
    'Task 4:"Imagine opening and closing fists or feet"': [6, 10, 14]
}

test_subjects = [34]
test_tasks = {
    'Task 1:"Open and close fist"': [3],
    'Task 2:"Imagine opening and closing fist"': [4],
    'Task 3:"Open and close fists or feet"': [5],
    'Task 4:"Imagine opening and closing fists or feet"': [6]
}

In [5]:
test_accuracy = []
cv_scores = []


for task_name in train_tasks:
    
    train_runs = train_tasks[task_name]
    test_runs = test_tasks[task_name]

    print(f'\nProcessing {task_name.split(":")[0]}...')

    train_epochs = load_and_preprocess_data(train_subjects, train_runs)
    X_train = train_epochs.get_data().astype('float64')
    y_train = train_epochs.events[:, -1]

    test_epochs = load_and_preprocess_data(test_subjects, test_runs)
    X_test = test_epochs.get_data().astype('float64')
    y_test = test_epochs.events[:, -1]

    X_train_flattened = X_train.reshape(X_train.shape[0], -1)
    X_test_flattened = X_test.reshape(X_test.shape[0], -1)

    test_acc, cv_sc = model_pipeline(X_train_flattened, y_train, X_test_flattened, y_test)

    print(f"\nExperiment {task_name.split(':')[1]}:")
    print(f"Test accuracy: {test_acc:.2f}, CV score: {cv_sc:.2f}")
    test_accuracy.append(test_acc)
    cv_scores.append(cv_sc)


print(f"\nMEAN:\nTest accuracy: {np.mean(test_accuracy):.2f}, CV score: {np.mean(cv_scores):.2f}")


Processing Task 1...

Experiment "Open and close fist":
Test accuracy: 0.77, CV score: 0.73

Processing Task 2...

Experiment "Imagine opening and closing fist":
Test accuracy: 0.71, CV score: 0.81

Processing Task 3...

Experiment "Open and close fists or feet":
Test accuracy: 0.68, CV score: 0.66

Processing Task 4...

Experiment "Imagine opening and closing fists or feet":
Test accuracy: 0.71, CV score: 0.70

MEAN:
Test accuracy: 0.72, CV score: 0.73
